In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import svm, tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
def read_csv_file (filename):
    df = pd.read_csv(filename)  # read the csv file
    return df

In [3]:
def convert_label_to_numeric(label):
    converted_label = np.empty(len(label), dtype=object) 
    for i in range(len(label)):
        if label[i] == "ONLY_ONE":
            converted_label[i] = 1
        elif label[i] == "NEUTRAL":
            converted_label[i] = 0
        else: 
            converted_label[i] = 2
    converted_label = converted_label.astype('int')
    return converted_label

In [4]:
def perfrom_CART(train, test, train_label, test_label):
    clf = tree.DecisionTreeClassifier(criterion='gini', splitter='best', 
                                      min_samples_split = 2, min_weight_fraction_leaf=0.0)
    clf.fit(train, train_label)
    predicted_label = clf.predict(test)
    recall, precision, f1 = measure_performance(test_label, predicted_label)
    return recall, precision, f1

In [5]:
def perfrom_KNN(train, test, train_label, test_label):
    clf = KNeighborsClassifier(n_neighbors=5, weights='distance')
    clf.fit(train, train_label)
    predicted_label = clf.predict(test)
    recall, precision, f1 = measure_performance(test_label, predicted_label)
    return recall, precision, f1

In [6]:
def perfrom_SVM(train, test, train_label, test_label):
    clf = svm.SVC(gamma='auto', C = 20.0, kernel='rbf', class_weight = {0:1, 1:1, 2:5})
    clf.fit(train, train_label)
    predicted_label = clf.predict(test)
    recall, precision, f1 = measure_performance(test_label, predicted_label)
    return recall, precision, f1

In [7]:
def perfrom_NB(train, test, train_label, test_label):
    clf = GaussianNB()
    clf.fit(train, train_label)
    predicted_label = clf.predict(test)
    recall, precision, f1 = measure_performance(test_label, predicted_label)
    return recall, precision, f1

In [8]:
def perfrom_RF(train, test, train_label, test_label):
    clf = RandomForestClassifier(n_estimators=10, criterion='gini')
    clf.fit(train, train_label)
    predicted_label = clf.predict(test)
    recall, precision, f1 = measure_performance(test_label, predicted_label)
    return recall, precision, f1

In [9]:
def measure_performance(true_label, predicted_label):   
    precision = recall = f1 = np.zeros(3, dtype=np.float32)
    report = classification_report(true_label, predicted_label, digits=3)
    precision[:] = precision_score(true_label, predicted_label, average=None, labels=[0,1,2])
    recall[:] = recall_score(true_label, predicted_label, average=None, labels=[0,1,2])
    f1[:] = f1_score(true_label, predicted_label, average=None, labels=[0,1,2])
    return recall, precision, f1

In [10]:
def kfold_cv(data, true_label):
    # 10 fold cv
    kf = KFold(n_splits=10, shuffle = True, random_state = 7)

    cv_recall_DT = []
    cv_precision_DT = []
    cv_f1_DT = []

    cv_recall_KNN = []
    cv_precision_KNN = []
    cv_f1_KNN = []

    cv_recall_SVM = []
    cv_precision_SVM = []
    cv_f1_SVM = []

    cv_recall_NB = []
    cv_precision_NB = []
    cv_f1_NB = []

    cv_recall_RF = []
    cv_precision_RF = []
    cv_f1_RF = []


    for train_index, test_index in kf.split(data):
        train, test = data.loc[train_index], data.loc[test_index]
        train_label, test_label = true_label[train_index], true_label[test_index]
    
        train = train.values.reshape(-1, 1)
        test = test.values.reshape(-1, 1)

        recall, precision, f1 = perfrom_CART(train, test, train_label, test_label)
        cv_recall_DT.append(recall)
        cv_precision_DT.append(precision)
        cv_f1_DT.append(f1)

        recall, precision, f1 = perfrom_KNN(train, test, train_label, test_label)
        cv_recall_KNN.append(recall)
        cv_precision_KNN.append(precision)
        cv_f1_KNN.append(f1)

        recall, precision, f1 = perfrom_SVM(train, test, train_label, test_label)
        cv_recall_SVM.append(recall)
        cv_precision_SVM.append(precision)
        cv_f1_SVM.append(f1)

        recall, precision, f1 = perfrom_NB(train, test, train_label, test_label)
        cv_recall_NB.append(recall)
        cv_precision_NB.append(precision)
        cv_f1_NB.append(f1)

        recall, precision, f1 = perfrom_RF(train, test, train_label, test_label)
        cv_recall_RF.append(recall)
        cv_precision_RF.append(precision)
        cv_f1_RF.append(f1)

    recall_DT = np.mean(cv_recall_DT, axis= 0)
    precision_DT = np.mean(cv_precision_DT, axis= 0)
    f1_DT = np.mean(cv_f1_DT, axis= 0)

    recall_KNN = np.mean(cv_recall_KNN, axis= 0)
    precision_KNN = np.mean(cv_precision_KNN, axis= 0)
    f1_KNN = np.mean(cv_f1_KNN, axis= 0)

    recall_SVM = np.mean(cv_recall_SVM, axis= 0)
    precision_SVM = np.mean(cv_precision_SVM, axis= 0)
    f1_SVM =  np.mean(cv_f1_SVM, axis= 0)

    recall_NB = np.mean(cv_recall_NB, axis= 0)
    precision_NB = np.mean(cv_precision_NB, axis= 0)
    f1_NB = np.mean(cv_f1_NB, axis= 0)

    recall_RF = np.mean(cv_recall_RF, axis= 0)
    precision_RF = np.mean(cv_precision_RF, axis= 0)
    f1_RF = np.mean(cv_f1_RF, axis= 0)
    
    return recall_DT, precision_DT, f1_DT, recall_KNN, precision_KNN, f1_KNN, recall_SVM, precision_SVM,\
    f1_SVM, recall_NB, precision_NB, f1_NB, recall_RF, precision_RF, f1_RF

In [11]:
def different_file_test(train, test, train_label, test_label):
    
    train = train.values.reshape(-1, 1)
    test = test.values.reshape(-1, 1)

    recall_DT, precision_DT, f1_DT = perfrom_CART(train, test, train_label, test_label)
    recall_KNN, precision_KNN, f1_KNN = perfrom_KNN(train, test, train_label, test_label)
    recall_SVM, precision_SVM, f1_SVM = perfrom_SVM(train, test, train_label, test_label)
    recall_NB, precision_NB, f1_NB = perfrom_NB(train, test, train_label, test_label)
    recall_RF, precision_RF, f1_RF = perfrom_RF(train, test, train_label, test_label)

    return recall_DT, precision_DT, f1_DT, recall_KNN, precision_KNN, f1_KNN, recall_SVM, precision_SVM,\
    f1_SVM, recall_NB, precision_NB, f1_NB, recall_RF, precision_RF, f1_RF

In [12]:
def repeated_test(data, true_label, train_data, test_data, train_label, test_label, test_name):
    repeated_recall_DT = []
    repeated_precision_DT = []
    repeated_f1_DT = []

    repeated_recall_KNN = []
    repeated_precision_KNN = []
    repeated_f1_KNN = []

    repeated_recall_SVM = []
    repeated_precision_SVM = []
    repeated_f1_SVM = []

    repeated_recall_NB = []
    repeated_precision_NB = []
    repeated_f1_NB = []

    repeated_recall_RF = []
    repeated_precision_RF = []
    repeated_f1_RF = []
    
    recall_DT= precision_DT= f1_DT= recall_KNN= precision_KNN= f1_KNN= recall_SVM= precision_SVM= f1_SVM\
    = recall_NB= precision_NB= f1_NB= recall_RF= precision_RF= f1_RF = 0
    
    for i in range(10):
        if test_name == "k_fold":
            recall_DT, precision_DT, f1_DT, recall_KNN, precision_KNN, f1_KNN, recall_SVM, precision_SVM, f1_SVM,\
            recall_NB, precision_NB, f1_NB, recall_RF, precision_RF, f1_RF = kfold_cv(data, true_label)
        else: 
            recall_DT, precision_DT, f1_DT, recall_KNN, precision_KNN, f1_KNN, recall_SVM, precision_SVM, f1_SVM,\
            recall_NB, precision_NB, f1_NB, recall_RF, precision_RF, f1_RF = different_file_test(train_data, test_data, train_label, test_label)
        
        repeated_recall_DT.append(recall_DT)
        repeated_precision_DT.append(precision_DT)
        repeated_f1_DT.append(f1_DT)

        repeated_recall_KNN.append(recall_KNN)
        repeated_precision_KNN.append(precision_KNN)
        repeated_f1_KNN.append(f1_KNN)

        repeated_recall_SVM.append(recall_SVM)
        repeated_precision_SVM.append(precision_SVM)
        repeated_f1_SVM.append(f1_SVM)

        repeated_recall_NB.append(recall_NB)
        repeated_precision_NB.append(precision_NB)
        repeated_f1_NB.append(f1_NB)

        repeated_recall_RF.append(recall_RF)
        repeated_precision_RF.append(precision_RF)
        repeated_f1_RF.append(f1_RF)
        
    print("-------DT-------")
    print("Recall:", np.median(repeated_recall_DT, axis= 0))
    print("Precision:", np.median(repeated_precision_DT, axis= 0))
    print("f1 score:", np.median(repeated_f1_DT, axis= 0))

    print("-------KNN-------")
    print("Recall:", np.median(repeated_recall_KNN, axis= 0))
    print("Precision:", np.median(repeated_precision_KNN, axis= 0))
    print("f1 score:", np.median(repeated_f1_KNN, axis= 0))

    print("-------SVM-------")
    print("Recall:", np.median(repeated_recall_SVM, axis= 0))
    print("Precision:", np.median(repeated_precision_SVM, axis= 0))
    print("f1 score:", np.median(repeated_f1_SVM, axis= 0))

    print("-------NB-------")
    print("Recall:", np.median(repeated_recall_NB, axis= 0))
    print("Precision:", np.median(repeated_precision_NB, axis= 0))
    print("f1 score:", np.median(repeated_f1_NB, axis= 0))

    print("-------RF-------")
    print("Recall:", np.median(repeated_recall_RF, axis= 0))
    print("Precision:", np.median(repeated_precision_RF, axis= 0))
    print("f1 score:", np.median(repeated_f1_RF, axis= 0))

In [13]:
data, true_label, train_data, test_data, train_label, test_label = [], [], [], [], [], [] # necessary variables

In [14]:
data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_MOZILLA.csv') 
print("Data set: COLOCATED_MOZILLA.csv")
print("Initial data shape: ", data.shape)
true_label = data['COLOCATED_STATUS']
true_label = convert_label_to_numeric(true_label)
data = data['SLOC']

repeated_test(data, true_label, train_data, test_data, train_label, test_label, "k_fold") #repeat kfold 10 times and report avarage performance

Data set: COLOCATED_MOZILLA.csv
Initial data shape:  (1613, 16)
-------DT-------
Recall: [0.9043578  0.08929107 0.30651155]
Precision: [0.9043578  0.08929107 0.30651155]
f1 score: [0.9043578  0.08929107 0.30651155]
-------KNN-------
Recall: [0.90055907 0.11625566 0.3304922 ]
Precision: [0.90055907 0.11625566 0.3304922 ]
f1 score: [0.90055907 0.11625566 0.3304922 ]
-------SVM-------
Recall: [0.84228075 0.02539683 0.34135723]
Precision: [0.84228075 0.02539683 0.34135723]
f1 score: [0.84228075 0.02539683 0.34135723]
-------NB-------
Recall: [0.9056792  0.         0.31992987]
Precision: [0.9056792  0.         0.31992987]
f1 score: [0.9056792  0.         0.31992987]
-------RF-------
Recall: [0.9060039  0.12049423 0.34554905]
Precision: [0.9060039  0.12049423 0.34554905]
f1 score: [0.9060039  0.12049423 0.34554905]


In [15]:
data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_OPENSTACK.csv') 
print("Data set: COLOCATED_OPENSTACK.csv")
print("Initial data shape: ", data.shape)
true_label = data['COLOCATED_STATUS']
true_label = convert_label_to_numeric(true_label)
data = data['SLOC']

repeated_test(data, true_label, train_data, test_data, train_label, test_label, "k_fold") #repeat kfold 10 times and report avarage performance

Data set: COLOCATED_OPENSTACK.csv
Initial data shape:  (2764, 16)
-------DT-------
Recall: [0.7980299  0.         0.37873894]
Precision: [0.7980299  0.         0.37873894]
f1 score: [0.7980299  0.         0.37873894]
-------KNN-------
Recall: [0.7849401  0.         0.39428034]
Precision: [0.7849401  0.         0.39428034]
f1 score: [0.7849401  0.         0.39428034]
-------SVM-------
Recall: [0.5854923 0.        0.5000353]
Precision: [0.5854923 0.        0.5000353]
f1 score: [0.5854923 0.        0.5000353]
-------NB-------
Recall: [0.81099254 0.         0.3066369 ]
Precision: [0.81099254 0.         0.3066369 ]
f1 score: [0.81099254 0.         0.3066369 ]
-------RF-------
Recall: [0.7955109  0.         0.40338424]
Precision: [0.7955109  0.         0.40338424]
f1 score: [0.7955109  0.         0.40338424]


In [16]:
data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_WIKIMEDIA.csv') 
print("Data set: COLOCATED_WIKIMEDIA.csv")
print("Initial data shape: ", data.shape)
true_label = data['COLOCATED_STATUS']
true_label = convert_label_to_numeric(true_label)
data = data['SLOC']

repeated_test(data, true_label, train_data, test_data, train_label, test_label, "k_fold") #repeat kfold 10 times and report avarage performance

Data set: COLOCATED_WIKIMEDIA.csv
Initial data shape:  (2845, 16)
-------DT-------
Recall: [0.8467064  0.         0.24716958]
Precision: [0.8467064  0.         0.24716958]
f1 score: [0.8467064  0.         0.24716958]
-------KNN-------
Recall: [0.83201665 0.00512821 0.2628052 ]
Precision: [0.83201665 0.00512821 0.2628052 ]
f1 score: [0.83201665 0.00512821 0.2628052 ]
-------SVM-------
Recall: [0.73785144 0.         0.42109329]
Precision: [0.73785144 0.         0.42109329]
f1 score: [0.73785144 0.         0.42109329]
-------NB-------
Recall: [0.8524454 0.        0.3122447]
Precision: [0.8524454 0.        0.3122447]
f1 score: [0.8524454 0.        0.3122447]
-------RF-------
Recall: [0.8422034  0.         0.29323125]
Precision: [0.8422034  0.         0.29323125]
f1 score: [0.8422034  0.         0.29323125]


In [17]:
train_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_MOZILLA.csv') 
print("Train Data: COLOCATED_MOZILLA.csv")
print("Initial data shape: ", train_data.shape)
train_label = train_data['COLOCATED_STATUS']
train_label = convert_label_to_numeric(train_label)
train_data = train_data['SLOC']

test_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_OPENSTACK.csv') 
print("Test Data: COLOCATED_OPENSTACK.csv")
print("Initial data shape: ", test_data.shape)
test_label = test_data['COLOCATED_STATUS']
test_label = convert_label_to_numeric(test_label)
test_data = test_data['SLOC']

repeated_test(data, true_label,train_data, test_data, train_label, test_label, "different_file_test") # repeat cross test 10 times and report avarage performance

Train Data: COLOCATED_MOZILLA.csv
Initial data shape:  (1613, 16)
Test Data: COLOCATED_OPENSTACK.csv
Initial data shape:  (2764, 16)
-------DT-------
Recall: [0.7913107  0.08163265 0.3642496 ]
Precision: [0.7913107  0.08163265 0.3642496 ]
f1 score: [0.7913107  0.08163265 0.3642496 ]
-------KNN-------
Recall: [0.7779441  0.08032128 0.37922895]
Precision: [0.7779441  0.08032128 0.37922895]
f1 score: [0.7779441  0.08032128 0.37922895]
-------SVM-------
Recall: [0.7080745  0.03529412 0.36253777]
Precision: [0.7080745  0.03529412 0.36253777]
f1 score: [0.7080745  0.03529412 0.36253777]
-------NB-------
Recall: [0.79313725 0.         0.39727065]
Precision: [0.79313725 0.         0.39727065]
f1 score: [0.79313725 0.         0.39727065]
-------RF-------
Recall: [0.77630603 0.09448448 0.3774824 ]
Precision: [0.77630603 0.09448448 0.3774824 ]
f1 score: [0.77630603 0.09448448 0.3774824 ]


In [18]:
train_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_MOZILLA.csv') 
print("Initial data shape: ", train_data.shape)
print("Train Data: COLOCATED_MOZILLA.csv")
train_label = train_data['COLOCATED_STATUS']
train_label = convert_label_to_numeric(train_label)
train_data = train_data['SLOC']

test_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_WIKIMEDIA.csv') 
print("Test Data: COLOCATED_WIKIMEDIA.csv")
print("Initial data shape: ", test_data.shape)
test_label = test_data['COLOCATED_STATUS']
test_label = convert_label_to_numeric(test_label)
test_data = test_data['SLOC']

repeated_test(data, true_label,train_data, test_data, train_label, test_label, "different_file_test") # repeat cross test 10 times and report avarage performance

Initial data shape:  (1613, 16)
Train Data: COLOCATED_MOZILLA.csv
Test Data: COLOCATED_WIKIMEDIA.csv
Initial data shape:  (2845, 16)
-------DT-------
Recall: [0.8505111  0.10108303 0.25941423]
Precision: [0.8505111  0.10108303 0.25941423]
f1 score: [0.8505111  0.10108303 0.25941423]
-------KNN-------
Recall: [0.846769   0.11147541 0.29023746]
Precision: [0.846769   0.11147541 0.29023746]
f1 score: [0.846769   0.11147541 0.29023746]
-------SVM-------
Recall: [0.78793633 0.08032128 0.33170733]
Precision: [0.78793633 0.08032128 0.33170733]
f1 score: [0.78793633 0.08032128 0.33170733]
-------NB-------
Recall: [0.8542735 0.        0.3462986]
Precision: [0.8542735 0.        0.3462986]
f1 score: [0.8542735 0.        0.3462986]
-------RF-------
Recall: [0.8458886  0.11218109 0.27774233]
Precision: [0.8458886  0.11218109 0.27774233]
f1 score: [0.8458886  0.11218109 0.27774233]


In [19]:
train_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_OPENSTACK.csv') 
print("Train Data: COLOCATED_OPENSTACK.csv")
print("Initial data shape: ", train_data.shape)
train_label = train_data['COLOCATED_STATUS']
train_label = convert_label_to_numeric(train_label)
train_data = train_data['SLOC']

test_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_MOZILLA.csv') 
print("Test Data: COLOCATED_MOZILLA.csv")
print("Initial data shape: ", test_data.shape)
test_label = test_data['COLOCATED_STATUS']
test_label = convert_label_to_numeric(test_label)
test_data = test_data['SLOC']

repeated_test(data, true_label,train_data, test_data, train_label, test_label, "different_file_test") # repeat cross test 10 times and report avarage performance

Train Data: COLOCATED_OPENSTACK.csv
Initial data shape:  (2764, 16)
Test Data: COLOCATED_MOZILLA.csv
Initial data shape:  (1613, 16)
-------DT-------
Recall: [0.8982036 0.        0.2614841]
Precision: [0.8982036 0.        0.2614841]
f1 score: [0.8982036 0.        0.2614841]
-------KNN-------
Recall: [0.8868061 0.        0.3045977]
Precision: [0.8868061 0.        0.3045977]
f1 score: [0.8868061 0.        0.3045977]
-------SVM-------
Recall: [0.7505679  0.         0.34056398]
Precision: [0.7505679  0.         0.34056398]
f1 score: [0.7505679  0.         0.34056398]
-------NB-------
Recall: [0.9039587  0.         0.16513762]
Precision: [0.9039587  0.         0.16513762]
f1 score: [0.9039587  0.         0.16513762]
-------RF-------
Recall: [0.8984219  0.         0.30769545]
Precision: [0.8984219  0.         0.30769545]
f1 score: [0.8984219  0.         0.30769545]


In [20]:
train_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_OPENSTACK.csv') 
print("Train Data: COLOCATED_OPENSTACK.csv")
print("Initial data shape: ", train_data.shape)
train_label = train_data['COLOCATED_STATUS']
train_label = convert_label_to_numeric(train_label)
train_data = train_data['SLOC']

test_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_WIKIMEDIA.csv') 
print("Test Data: COLOCATED_WIKIMEDIA.csv")
print("Initial data shape: ", test_data.shape)
test_label = test_data['COLOCATED_STATUS']
test_label = convert_label_to_numeric(test_label)
test_data = test_data['SLOC']

repeated_test(data, true_label,train_data, test_data, train_label, test_label, "different_file_test") # repeat cross test 10 times and report avarage performance

Train Data: COLOCATED_OPENSTACK.csv
Initial data shape:  (2764, 16)
Test Data: COLOCATED_WIKIMEDIA.csv
Initial data shape:  (2845, 16)
-------DT-------
Recall: [0.8414608 0.        0.2146739]
Precision: [0.8414608 0.        0.2146739]
f1 score: [0.8414608 0.        0.2146739]
-------KNN-------
Recall: [0.8285466 0.        0.2622577]
Precision: [0.8285466 0.        0.2622577]
f1 score: [0.8285466 0.        0.2622577]
-------SVM-------
Recall: [0.7016559  0.         0.41400942]
Precision: [0.7016559  0.         0.41400942]
f1 score: [0.7016559  0.         0.41400942]
-------NB-------
Recall: [0.8518212 0.        0.1891473]
Precision: [0.8518212 0.        0.1891473]
f1 score: [0.8518212 0.        0.1891473]
-------RF-------
Recall: [0.8403485  0.00907034 0.25064638]
Precision: [0.8403485  0.00907034 0.25064638]
f1 score: [0.8403485  0.00907034 0.25064638]


In [21]:
train_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_WIKIMEDIA.csv') 
print("Train Data: COLOCATED_WIKIMEDIA.csv")
print("Initial data shape: ", train_data.shape)
train_label = train_data['COLOCATED_STATUS']
train_label = convert_label_to_numeric(train_label)
train_data = train_data['SLOC']

test_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_MOZILLA.csv') 
print("Test Data: COLOCATED_MOZILLA.csv")
print("Initial data shape: ", test_data.shape)
test_label = test_data['COLOCATED_STATUS']
test_label = convert_label_to_numeric(test_label)
test_data = test_data['SLOC']

repeated_test(data, true_label,train_data, test_data, train_label, test_label, "different_file_test") # repeat cross test 10 times and report avarage performance

Train Data: COLOCATED_WIKIMEDIA.csv
Initial data shape:  (2845, 16)
Test Data: COLOCATED_MOZILLA.csv
Initial data shape:  (1613, 16)
-------DT-------
Recall: [0.9051815  0.05405406 0.2877698 ]
Precision: [0.9051815  0.05405406 0.2877698 ]
f1 score: [0.9051815  0.05405406 0.2877698 ]
-------KNN-------
Recall: [0.8952449  0.07207207 0.3207547 ]
Precision: [0.8952449  0.07207207 0.3207547 ]
f1 score: [0.8952449  0.07207207 0.3207547 ]
-------SVM-------
Recall: [0.7651183  0.         0.32142857]
Precision: [0.7651183  0.         0.32142857]
f1 score: [0.7651183  0.         0.32142857]
-------NB-------
Recall: [0.90510947 0.         0.2682927 ]
Precision: [0.90510947 0.         0.2682927 ]
f1 score: [0.90510947 0.         0.2682927 ]
-------RF-------
Recall: [0.90102863 0.01826827 0.29897833]
Precision: [0.90102863 0.01826827 0.29897833]
f1 score: [0.90102863 0.01826827 0.29897833]


In [22]:
train_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_WIKIMEDIA.csv') 
print("Train Data: COLOCATED_WIKIMEDIA.csv")
print("Initial data shape: ", train_data.shape)
train_label = train_data['COLOCATED_STATUS']
train_label = convert_label_to_numeric(train_label)
train_data = train_data['SLOC']

test_data = pd.read_csv('..//updated_data//RAW_DATASETS//COLOCATED_OPENSTACK.csv') 
print("Test Data: COLOCATED_OPENSTACK.csv")
print("Initial data shape: ", test_data.shape)
test_label = test_data['COLOCATED_STATUS']
test_label = convert_label_to_numeric(test_label)
test_data = test_data['SLOC']

repeated_test(data, true_label,train_data, test_data, train_label, test_label, "different_file_test") # repeat cross test 10 times and report avarage performance

Train Data: COLOCATED_WIKIMEDIA.csv
Initial data shape:  (2845, 16)
Test Data: COLOCATED_OPENSTACK.csv
Initial data shape:  (2764, 16)
-------DT-------
Recall: [0.7855247  0.07185629 0.32401314]
Precision: [0.7855247  0.07185629 0.32401314]
f1 score: [0.7855247  0.07185629 0.32401314]
-------KNN-------
Recall: [0.7879234  0.04469274 0.37019607]
Precision: [0.7879234  0.04469274 0.37019607]
f1 score: [0.7879234  0.04469274 0.37019607]
-------SVM-------
Recall: [0.55932206 0.02857143 0.48748043]
Precision: [0.55932206 0.02857143 0.48748043]
f1 score: [0.55932206 0.02857143 0.48748043]
-------NB-------
Recall: [0.7968147  0.         0.37609562]
Precision: [0.7968147  0.         0.37609562]
f1 score: [0.7968147  0.         0.37609562]
-------RF-------
Recall: [0.7782166  0.05571074 0.35781196]
Precision: [0.7782166  0.05571074 0.35781196]
f1 score: [0.7782166  0.05571074 0.35781196]
